In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, StandardScaler
import numpy as np

In [2]:
data = pd.read_csv('../dataset/user-data-cleaned.csv')

In [3]:
encoder = OneHotEncoder()
multilabel_binarizer = MultiLabelBinarizer()
scaler = StandardScaler()

In [4]:
encoded_education = encoder.fit_transform(data[['currentEducation']])
encoded_language = encoder.fit_transform(data[['language']])

In [5]:
encoded_interests = multilabel_binarizer.fit_transform(data['interest_list'])
encoded_skills = multilabel_binarizer.fit_transform(data['skill_list'])
encoded_learning_resources = multilabel_binarizer.fit_transform(data['learningResource_list'])
encoded_tools = multilabel_binarizer.fit_transform(data['tools_list'])

In [6]:
normalized_experience = scaler.fit_transform(data[['experienceYears']])

In [7]:
user_features = np.hstack((
    normalized_experience,
    encoded_education,
    encoded_language,
    encoded_interests,
    encoded_skills,
    encoded_learning_resources,
    encoded_tools
))

# Final preprocessed data
print(user_features)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)